# FedPSO (clientes com dados completos)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms
import copy
import random
import csv
import torchvision.models as models

In [2]:
#Hiperparâmetros
BATCH_SIZE = 64
epocas_cliente = 5
num_clientes = 10
num_rodadas = 30
inertia, c1, c2 = 0.9, 0.8, 0.9
learning_rate = 0.0001

In [3]:
# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#Seeds para reprodutibilidade
random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# Função para inicializar o modelo de acordo com a arquitetura
def init_model(architecture="squeezenet"):
    if architecture == "squeezenet":
        model = models.squeezenet1_1(weights=None)
        # model = models.squeezenet1_1(weights=models.SqueezeNet1_1_Weights.IMAGENET1K_V1)
        model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1))
    elif architecture == "mobilenet_v2":
        model = models.mobilenet_v2(weights=None)
        # model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
    elif architecture == "resnet18":
        model = models.resnet18(weights=None)
        # model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        model.fc = nn.Linear(model.fc.in_features, 10)
    elif architecture == "alexnet":
        model = models.alexnet(weights=None)
         # model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    else:
        raise ValueError("Unsupported architecture. Choose from 'squeezenet', 'mobilenet_v2', 'resnet18', 'alexnet'.")
    return model.to(device)

# Função para criar um subconjunto aleatório de um dataset
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)

# Função para salvar os resultados em um arquivo CSV
def write_csv(algorithm_name, architecture, results):
    file_name = f'{algorithm_name}_{architecture}_do_zero_CIFAR10_output.csv'
    with open(file_name, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Epoch", "Test Loss", "Test Accuracy"])
        for i, result in enumerate(results):
            writer.writerow([i + 1] + result)

class Particle:
    def __init__(self, particle_id, model, data, architecture="squeezenet"):
        self.particle_id = particle_id
        self.model = model.to(device)
        self.data = data  # `self.data` agora armazena o dataset, não o DataLoader
        self.local_best_model = copy.deepcopy(model.state_dict())
        self.local_best_score = float('inf')
        self.velocity = {name: torch.zeros_like(param) for name, param in model.named_parameters()}
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001, weight_decay=1e-5)

    def train_particle(self, criterion):
        self.model.train()
        train_loader = DataLoader(self.data, batch_size=BATCH_SIZE, shuffle=True)
        for _ in range(epocas_cliente):
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

        val_loss = self.evaluate_loss(criterion)
        if val_loss < self.local_best_score:
            self.local_best_score = val_loss
            self.local_best_model = copy.deepcopy(self.model.state_dict())
        return self.particle_id, val_loss

    def evaluate_loss(self, criterion):
        self.model.eval()
        total_loss = 0
        val_loader = DataLoader(self.data, batch_size=BATCH_SIZE, shuffle=False)
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
        return total_loss / len(val_loader)

    def pso_update(self, global_best_model, inertia, c1, c2,learning_rate):
        for name, param in self.model.named_parameters():
            if param.grad is None:
                continue
            local_rand = random.random()
            global_rand = random.random()
            self.velocity[name] = (
                inertia * self.velocity[name]
                + c1 * local_rand * (self.local_best_model[name].to(device) - param.data)
                + c2 * global_rand * (global_best_model[name].to(device) - param.data)
            )
            param.data += self.velocity[name]
            param.data = param.data - learning_rate * param.grad

def train_federated(architecture="squeezenet"):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloaders = [create_subset(trainset, 50000) for _ in range(num_clientes)]

    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

    server_model = init_model(architecture)
    particles = [Particle(i, copy.deepcopy(server_model), trainloaders[i], architecture) for i in range(num_clientes)]
    criterion = nn.CrossEntropyLoss()

    global_best_score = float('inf')
    global_best_model = copy.deepcopy(server_model.state_dict())
    server_evaluate_acc = []

    for epoch in range(num_rodadas):
        server_result = []
        for particle in particles:
            if epoch > 0:
                particle.pso_update(global_best_model, inertia, c1, c2, learning_rate)
            pid, val_loss = particle.train_particle(criterion)
            server_result.append((pid, val_loss))

        best_particle = min(server_result, key=lambda x: x[1])
        global_best_score = best_particle[1]
        global_best_model = copy.deepcopy(particles[best_particle[0]].local_best_model)

        print(f'Epoch {epoch + 1}/{num_rodadas}, Best Particle Loss: {global_best_score:.4f}')

        server_model.load_state_dict(global_best_model)
        server_model.eval()
        total_loss, correct = 0, 0
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                outputs = server_model(images)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
        test_loss = total_loss / len(testloader)
        test_accuracy = correct / len(testset)
        server_evaluate_acc.append([test_loss, test_accuracy])
        print(f'Epoch {epoch + 1}/{num_rodadas}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

    write_csv("FLPSO-SGD3", architecture, server_evaluate_acc)

# Exemplo de execução
#train_federated("squeezenet")

## SqueezeNet

In [5]:
train_federated("squeezenet")  

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Best Particle Loss: 1.1010
Epoch 1/30, Test Loss: 1.1292, Test Accuracy: 0.5908
Epoch 2/30, Best Particle Loss: 0.8720
Epoch 2/30, Test Loss: 0.9242, Test Accuracy: 0.6735
Epoch 3/30, Best Particle Loss: 0.7320
Epoch 3/30, Test Loss: 0.8138, Test Accuracy: 0.7174
Epoch 4/30, Best Particle Loss: 0.6360
Epoch 4/30, Test Loss: 0.7464, Test Accuracy: 0.7407
Epoch 5/30, Best Particle Loss: 0.5279
Epoch 5/30, Test Loss: 0.6657, Test Accuracy: 0.7726
Epoch 6/30, Best Particle Loss: 0.4647
Epoch 6/30, Test Loss: 0.6604, Test Accuracy: 0.7795
Epoch 7/30, Best Particle Loss: 0.3897
Epoch 7/30, Test Loss: 0.6211, Test Accuracy: 0.7914
Epoch 8/30, Best Particle Loss: 0.3327
Epoch 8/30, Test Loss: 0.6418, Test Accuracy: 0.7935
Epoch 9/30, Best Particle Loss: 0.2694
Epoch 9/30, Test Loss: 0.6152, Test Accuracy: 0.8001
Epoch 10/30, Best Particle Loss: 0.2180
Epoch 10/30, Test Loss: 0.6649, Test Accuracy: 0.8011
Ep

## MobileNet_v2

In [ ]:
train_federated("mobilenet_v2") 

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Best Particle Loss: 0.6979
Epoch 1/30, Test Loss: 0.8544, Test Accuracy: 0.6993
Epoch 2/30, Best Particle Loss: 0.2832
Epoch 2/30, Test Loss: 0.7297, Test Accuracy: 0.7570
Epoch 3/30, Best Particle Loss: 0.0889
Epoch 3/30, Test Loss: 0.8571, Test Accuracy: 0.7616
Epoch 4/30, Best Particle Loss: 0.0477
Epoch 4/30, Test Loss: 0.8438, Test Accuracy: 0.7795
Epoch 5/30, Best Particle Loss: 0.0307
Epoch 5/30, Test Loss: 0.8464, Test Accuracy: 0.7910
Epoch 6/30, Best Particle Loss: 0.0263
Epoch 6/30, Test Loss: 0.9428, Test Accuracy: 0.7897
Epoch 7/30, Best Particle Loss: 0.0168
Epoch 7/30, Test Loss: 0.9681, Test Accuracy: 0.7902
Epoch 8/30, Best Particle Loss: 0.0154
Epoch 8/30, Test Loss: 0.9565, Test Accuracy: 0.7974
Epoch 9/30, Best Particle Loss: 0.0120
Epoch 9/30, Test Loss: 1.0132, Test Accuracy: 0.7985
Epoch 10/30, Best Particle Loss: 0.0131
Epoch 10/30, Test Loss: 0.9978, Test Accuracy: 0.7962
Ep

## Resnet18

In [ ]:
train_federated("resnet18") 

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Best Particle Loss: 0.2234
Epoch 1/30, Test Loss: 0.6456, Test Accuracy: 0.7845
Epoch 2/30, Best Particle Loss: 0.0829
Epoch 2/30, Test Loss: 0.8104, Test Accuracy: 0.7763
Epoch 3/30, Best Particle Loss: 0.0458
Epoch 3/30, Test Loss: 0.7826, Test Accuracy: 0.7989
Epoch 4/30, Best Particle Loss: 0.0407
Epoch 4/30, Test Loss: 0.8499, Test Accuracy: 0.8113
Epoch 5/30, Best Particle Loss: 0.0262
Epoch 5/30, Test Loss: 0.8649, Test Accuracy: 0.8125


## AlexNet

In [ ]:
train_federated("alexnet") 